# End-End RAG Backend Implementation

Disclaimer: I used ChatGPT for some parts and marked prompts where I used it.

## Libraries

In [2]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.1 MB/s eta 0:00

In [3]:
!pip install pdfplumber transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 39.7 MB/s eta 0:00:00


In [4]:
import json
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer

In [6]:
import shutil
import pandas as pd

In [57]:
import os
from dotenv import load_dotenv

In [8]:
import google.generativeai as genai

## Document Preparation

In [9]:
json_path = "/content/uchicago_ads_scraped.json"

with open(json_path, "r", encoding="utf-8") as f:
    scraped_data = json.load(f)

print(f"Loaded {len(scraped_data)} pages.")

Loaded 66 pages.


In [30]:
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
urls = list(scraped_data.keys())
texts = list(scraped_data.values())

In [32]:
embeddings = model.encode(texts, show_progress_bar=True)

print(f"Generated embeddings shape: {embeddings.shape}")

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Generated embeddings shape: (66, 768)


In [33]:
sample_idx = 0
print(f"URL: {urls[sample_idx]}\n")
print(f"Text sample: {texts[sample_idx][:500]}...\n")
print(f"Embedding vector (first 10 values):\n{embeddings[sample_idx][:10]}")

URL: https://datascience.uchicago.edu/news/can-a-doctors-notes-reveal-when-theyre-tired-new-research-illuminates-the-hidden-signals-of-physician-fatigue-and-raises-questions-about-ai-in-healthcare/

Text sample: Can a Doctor’s Notes Reveal When They’re Tired? New Research Illuminates the Hidden Signals of Physician Fatigue—And Raises Questions About AI in Healthcare – DSI Skip to main content About About the Data Science Institute The Data Science Institute (DSI) executes the University of Chicago’s bold, innovative vision of Data Science as a new discipline. Jobs & Opportunities Open faculty, postdoctoral, staff, and student roles with the UChicago Data Science Institute and our partners. Visiting DSI ...

Embedding vector (first 10 values):
[ 1.0165022e-02  1.2713394e-01 -5.5859935e-02 -5.6162953e-02
 -2.7033484e-02  4.5889985e-02  5.0503403e-02 -1.8186683e-03
  1.9919192e-02  5.5047130e-06]


In [34]:
# combine one df with urls and texts
df = pd.DataFrame({
    'url': list(scraped_data.keys()),
    'text': list(scraped_data.values()),
    'embedding': list(embeddings)
})

print(df.head())

                                                 url  \
0  https://datascience.uchicago.edu/news/can-a-do...   
1         https://datascience.uchicago.edu/outreach/   
2  https://datascience.uchicago.edu/insights/nove...   
3  https://datascience.uchicago.edu/outreach/11th...   
4  https://datascience.uchicago.edu/events/ask-a-...   

                                                text  \
0  Can a Doctor’s Notes Reveal When They’re Tired...   
1  Outreach – DSI Skip to main content About Abou...   
2  Novel Intelligence Faculty Co-Director James E...   
3  11th Hour Project Software and Data Hub – DSI ...   
4  Ask A Student Information Session (Online Stud...   

                                           embedding  
0  [0.010165022, 0.12713394, -0.055859935, -0.056...  
1  [-0.014629674, 0.12732139, -0.066904664, -0.03...  
2  [0.016023358, 0.13400435, -0.06181596, -0.0316...  
3  [-0.0078848945, 0.116841026, -0.064854145, -0....  
4  [-0.0060235215, 0.12022137, -0.062113486, -0.0..

##  Store the Embeddings in a local Chroma vector database

AI Prompt: "I have a data frame with 3 columns: url, text (from the url), and embeddings. I want to embed them in a ChromaDB. how can I do this?"

In [35]:
# Create LangChain documents from your DataFrame
documents = [
    Document(
        page_content=row['text'],
        metadata={'source': row['url']}
    )
    for _, row in df.iterrows()
]

In [36]:
documents[0]

Document(metadata={'source': 'https://datascience.uchicago.edu/news/can-a-doctors-notes-reveal-when-theyre-tired-new-research-illuminates-the-hidden-signals-of-physician-fatigue-and-raises-questions-about-ai-in-healthcare/'}, page_content="Can a Doctor’s Notes Reveal When They’re Tired? New Research Illuminates the Hidden Signals of Physician Fatigue—And Raises Questions About AI in Healthcare – DSI Skip to main content About About the Data Science Institute The Data Science Institute (DSI) executes the University of Chicago’s bold, innovative vision of Data Science as a new discipline. Jobs & Opportunities Open faculty, postdoctoral, staff, and student roles with the UChicago Data Science Institute and our partners. Visiting DSI @ UChicago Contact Research Initiatives AI + Science A new paradigm of transformational AI-enabled scientific discovery across the physical and biological sciences. Data & Democracy Protecting democracy in the digital age through cross-disciplinary research an

In [38]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [40]:
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_uchicago_ads2"
)


In [51]:
# make zip file to save for reuse later if needed
shutil.make_archive("/content/chroma_uchicago_ads2", 'zip', "/content/chroma_uchicago_ads2")


'/content/chroma_uchicago_ads2.zip'

## Query Response Pipeline

### Load Gemini Model

In [69]:
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [71]:
genai.configure(api_key=gemini_api_key)

### Create RAG

In [41]:
def RAG_pipeline_gemini(query, db, k=5):
    # Use the retriever to get top-k relevant chunks from Chroma
    # NOTE: The retriever automatically converts the query into an embedding using
    # the same encoder that was used for storing the document chunks.
    # No manual embedding needed.
    retriever = db.as_retriever(search_kwargs={"k": k})
    retrieved_docs = retriever.get_relevant_documents(query)

    # Combine the retrieved context into a single block of text
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

      # Compose the RAG prompt by combining the context and the user's question
    rag_prompt = f"""You are a helpful assistant. Use the following context to answer the question.

[CONTEXT]
{context}

[QUESTION]
{query}

Answer:"""

    model = genai.GenerativeModel("models/gemini-2.5-pro")

    response = model.generate_content(rag_prompt)

    # Print results + sources
    print("\n=== Question ===")
    print(query)
    print("\n=== Generated Answer ===")
    print(response.text)

    print("\n=== Sources ===")
    for doc in retrieved_docs:
        print(f"- {doc.metadata.get('source', 'No source available')}")

    return response.text


## Test Responses

In [42]:
RAG_pipeline_gemini("What are the admission requirements for the UChicago ADS program?", db)


=== Question ===
What are the admission requirements for the UChicago ADS program?

=== Generated Answer ===
Based on the context provided, here are the admission requirements for the Master's in Applied Data Science (MS-ADS) program at the University of Chicago:

*   **Letters of Recommendation:** Two letters are required. It is strongly recommended that at least one letter comes from a professional source like a direct manager or internship supervisor. Professors and professional mentors are also acceptable sources. Letters from family, friends, or peers are not accepted.
*   **Candidate Statement:** A statement of no more than 1,000 words that outlines your aims and qualifications. The application will provide a specific prompt.
*   **Resume/CV:** An updated resume or CV that includes work experience, major accomplishments, educational background, and professional conferences attended.
*   **Programming Supplement:** A PDF (no more than two pages) that demonstrates your current ski

"Based on the context provided, here are the admission requirements for the Master's in Applied Data Science (MS-ADS) program at the University of Chicago:\n\n*   **Letters of Recommendation:** Two letters are required. It is strongly recommended that at least one letter comes from a professional source like a direct manager or internship supervisor. Professors and professional mentors are also acceptable sources. Letters from family, friends, or peers are not accepted.\n*   **Candidate Statement:** A statement of no more than 1,000 words that outlines your aims and qualifications. The application will provide a specific prompt.\n*   **Resume/CV:** An updated resume or CV that includes work experience, major accomplishments, educational background, and professional conferences attended.\n*   **Programming Supplement:** A PDF (no more than two pages) that demonstrates your current skills in a programming language like R or Python. The committee looks for examples of ingesting data, data

In [45]:
RAG_pipeline_gemini("What are the core courses UChicago In-person ADS program?", db)


=== Question ===
What are the core courses UChicago In-person ADS program?

=== Generated Answer ===
Based on the context provided, students in the University of Chicago's in-person MS in Applied Data Science program must complete 6 core courses. The core courses are:

*   Time Series Analysis and Forecasting
*   Statistical Models for Data Science
*   Machine Learning I
*   Machine Learning II
*   Data Engineering Platforms for Analytics **or** Big Data and Cloud Computing
*   Leadership and Consulting for Data Science

=== Sources ===
- https://datascience.uchicago.edu/education/undergrad-major/
- https://datascience.uchicago.edu/education/masters-programs/
- https://datascience.uchicago.edu/education/masters-programs/online-program/
- https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/online-program/%20
- https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/in-person-program/


"Based on the context provided, students in the University of Chicago's in-person MS in Applied Data Science program must complete 6 core courses. The core courses are:\n\n*   Time Series Analysis and Forecasting\n*   Statistical Models for Data Science\n*   Machine Learning I\n*   Machine Learning II\n*   Data Engineering Platforms for Analytics **or** Big Data and Cloud Computing\n*   Leadership and Consulting for Data Science"

In [47]:
RAG_pipeline_gemini("What are the core courses UChicago ADS program?", db)


=== Question ===
What are the core courses UChicago ADS program?

=== Generated Answer ===
Based on the context provided, the University of Chicago's Master's in Applied Data Science (MS-ADS) program requires students to complete 6 core courses. The core courses are:

*   Time Series Analysis and Forecasting
*   Statistical Models for Data Science
*   Machine Learning I
*   Machine Learning II
*   Data Engineering Platforms for Analytics **or** Big Data and Cloud Computing
*   Leadership and Consulting for Data Science

In addition to these 6 core courses, students must also complete 4 elective courses and a 2-quarter required Capstone Project to earn their degree.

=== Sources ===
- https://datascience.uchicago.edu/education/undergrad-major/
- https://datascience.uchicago.edu/education/masters-programs/
- https://datascience.uchicago.edu/education/masters-programs/online-program/
- https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/online-program/%

"Based on the context provided, the University of Chicago's Master's in Applied Data Science (MS-ADS) program requires students to complete 6 core courses. The core courses are:\n\n*   Time Series Analysis and Forecasting\n*   Statistical Models for Data Science\n*   Machine Learning I\n*   Machine Learning II\n*   Data Engineering Platforms for Analytics **or** Big Data and Cloud Computing\n*   Leadership and Consulting for Data Science\n\nIn addition to these 6 core courses, students must also complete 4 elective courses and a 2-quarter required Capstone Project to earn their degree."

In [48]:
RAG_pipeline_gemini("What is the difference between the UChicago ADS in person and online programs?", db)


=== Question ===
What is the difference between the UChicago ADS in person and online programs?

=== Generated Answer ===
Based on the context provided, here are the key differences between the University of Chicago's in-person and online MS in Applied Data Science programs:

**Format and Location:**
*   **In-Person Program:** Classes are held in person in downtown Chicago at the NBC Tower or Gleacher Center. It is designed for students who prefer a traditional classroom environment and access to in-person services.
*   **Online Program:** The program is delivered remotely with a combination of weekly, synchronous (real-time) classes and asynchronous (self-paced) content. Live classes are scheduled on weekday evenings and occasional Saturdays. It also offers a unique, optional in-person "Immersion Weekend" in Chicago during the Autumn quarter for networking.

**Program Options and Duration:**
*   **In-Person Program:** Offers two pathways:
    1.  A **1-year, 12-course program** (6 co

'Based on the context provided, here are the key differences between the University of Chicago\'s in-person and online MS in Applied Data Science programs:\n\n**Format and Location:**\n*   **In-Person Program:** Classes are held in person in downtown Chicago at the NBC Tower or Gleacher Center. It is designed for students who prefer a traditional classroom environment and access to in-person services.\n*   **Online Program:** The program is delivered remotely with a combination of weekly, synchronous (real-time) classes and asynchronous (self-paced) content. Live classes are scheduled on weekday evenings and occasional Saturdays. It also offers a unique, optional in-person "Immersion Weekend" in Chicago during the Autumn quarter for networking.\n\n**Program Options and Duration:**\n*   **In-Person Program:** Offers two pathways:\n    1.  A **1-year, 12-course program** (6 core, 4 electives, 2 Capstone).\n    2.  A **2-year, 18-course Thesis Track program** (full-time only) that include

In [49]:
RAG_pipeline_gemini("What programs does the UChicago DSI offer?", db)


=== Question ===
What programs does the UChicago DSI offer?

=== Generated Answer ===
Based on the context provided, the UChicago Data Science Institute (DSI) offers the following programs:

**Academic Programs:**

*   **Undergraduate Data Science Major:** A curriculum that combines computational and analytical skills, domain knowledge, communication, and ethics.
*   **Master's Programs:**
    *   **MS in Applied Data Science:** Designed for professionals with technical backgrounds, offered in both in-person and online formats.
    *   **MS in Data Science (MSDS):** For students interested in a research career in data science.
    *   **MS in Computational Analysis and Public Policy:** A two-year program offered jointly with the Harris School of Public Policy.
    *   **MBA/MS in Applied Data Science:** A joint degree program at the intersection of business and technology.
*   **PhD in Data Science:** A program combining training in mathematical foundations, responsible data use, and 

"Based on the context provided, the UChicago Data Science Institute (DSI) offers the following programs:\n\n**Academic Programs:**\n\n*   **Undergraduate Data Science Major:** A curriculum that combines computational and analytical skills, domain knowledge, communication, and ethics.\n*   **Master's Programs:**\n    *   **MS in Applied Data Science:** Designed for professionals with technical backgrounds, offered in both in-person and online formats.\n    *   **MS in Data Science (MSDS):** For students interested in a research career in data science.\n    *   **MS in Computational Analysis and Public Policy:** A two-year program offered jointly with the Harris School of Public Policy.\n    *   **MBA/MS in Applied Data Science:** A joint degree program at the intersection of business and technology.\n*   **PhD in Data Science:** A program combining training in mathematical foundations, responsible data use, and advanced computational methods.\n*   **Postdoctoral Programs:** Opportunitie

In [50]:
RAG_pipeline_gemini("Can you provide information about the capstone project?", db)


=== Question ===
Can you provide information about the capstone project?

=== Generated Answer ===
Based on the context provided, here is information about the Capstone Project at the University of Chicago's Data Science Institute (DSI):

The Capstone Project is the culminating experience for students in the Master’s in Applied Data Science (MS-ADS) program. It allows students to apply their knowledge and skills to a real-world business problem using authentic data.

**Key Features of the Capstone Project:**

*   **Real-World Experience:** Students work in faculty-mentored teams on projects with real clients (sponsors) from various sectors, including finance, entertainment, and automotive. They also have the option to join a research-focused team.
*   **Skill Enhancement:** The project is designed to enhance students' collaboration skills and provides them with mentoring and networking opportunities.
*   **Value for Sponsors:** Sponsoring organizations propose real-world problems or r

"Based on the context provided, here is information about the Capstone Project at the University of Chicago's Data Science Institute (DSI):\n\nThe Capstone Project is the culminating experience for students in the Master’s in Applied Data Science (MS-ADS) program. It allows students to apply their knowledge and skills to a real-world business problem using authentic data.\n\n**Key Features of the Capstone Project:**\n\n*   **Real-World Experience:** Students work in faculty-mentored teams on projects with real clients (sponsors) from various sectors, including finance, entertainment, and automotive. They also have the option to join a research-focused team.\n*   **Skill Enhancement:** The project is designed to enhance students' collaboration skills and provides them with mentoring and networking opportunities.\n*   **Value for Sponsors:** Sponsoring organizations propose real-world problems or research queries. In return, they gain access to students and faculty, receive actionable in

In [88]:
RAG_pipeline_gemini("What is the career seminar course for ADS", db)


=== Question ===
What is the career seminar course for ADS

=== Generated Answer ===
Based on the context provided, the Career Seminar for the MS in Applied Data Science (ADS) program is a required, noncredit, pass/fail course. Here are the key details:

*   **Purpose:** It supports the development of professional skills for the industry, assists with job and internship searches, and covers other in-demand competencies sought by employers.
*   **Structure:** Students enroll in the seminar each quarter to engage with new content throughout their degree program.
*   **Cost and Credit:** It is a noncredit course (0 units) and has no cost.
*   **Waiver:** Students with significant full-time work experience (3+ years) may be eligible to waive this course requirement.

=== Sources ===
- https://datascience.uchicago.edu/about/jobs/
- https://datascience.uchicago.edu/education/undergrad-major/
- https://datascience.uchicago.edu/education/masters-programs/
- https://datascience.uchicago.edu/ed

'Based on the context provided, the Career Seminar for the MS in Applied Data Science (ADS) program is a required, noncredit, pass/fail course. Here are the key details:\n\n*   **Purpose:** It supports the development of professional skills for the industry, assists with job and internship searches, and covers other in-demand competencies sought by employers.\n*   **Structure:** Students enroll in the seminar each quarter to engage with new content throughout their degree program.\n*   **Cost and Credit:** It is a noncredit course (0 units) and has no cost.\n*   **Waiver:** Students with significant full-time work experience (3+ years) may be eligible to waive this course requirement.'

## Metrics

Calculating response relevance based on key words in a test set.

In [74]:
# test set with questions and expected keywords (questions from assignment PDF)
test_set = [
    {
        "question": "What are the admission requirements for the UChicago ADS program?",
        "expected_keywords": ["fee", "application", "transcript", "video", "resume",
                              "recommendation", "CV", "bachelor", "degree", "coursework",
                              "mathematics", "statistics", "essay", "statement", "TOEFL",
                              "english", "programming"]
    },
    {
        "question": "What are the core courses in the UChicago ADS program?",
        "expected_keywords": ["Machine Learning", "Time Series", "Forecasting",
                              "Data Engineering", "Capstone", "Statistical Models",
                              "Big Data", "Cloud Computing", "Leadership", "Consulting"]
    },
    {
        "question": "Can you provide information about the capstone project?",
        "expected_keywords": ["team", "industry", "research", "presentation",
                              "proposal", "client", "real-world", "data", "solution"]
    }
]

In [84]:
# get the accuracy score based on how many key words were present in response
def get_overall_accuracy(rag_function, test_set, db):
    total_score = 0
    len_kw = 0
    for test in test_set:
        answer = rag_function(test["question"], db).lower()
        found = sum(1 for kw in test["expected_keywords"] if kw.lower() in answer)
        total_score += found
        len_kw += len(test["expected_keywords"])

    return total_score / len_kw

accuracy = get_overall_accuracy(RAG_pipeline_gemini, test_set, db)
print(f"Overall RAG accuracy: {accuracy:.1%}")


=== Question ===
What are the admission requirements for the UChicago ADS program?

=== Generated Answer ===
Based on the context provided, here are the admission requirements for the Master’s in Applied Data Science (MS-ADS) program at the University of Chicago:

*   **Letters of Recommendation:** Two letters are required. It is strongly recommended that at least one is from a professional source like a direct manager or internship supervisor.
*   **Candidate Statement:** A statement of up to 1,000 words. The application will provide a specific prompt.
*   **Resume/CV:** An updated resume detailing work experience, major accomplishments, awards, and educational background.
*   **Programming Supplement:** A PDF of no more than two pages demonstrating programming skills in a language like R or Python. The supplement should showcase abilities such as data ingestion, data wrangling, visualization, and function writing.
*   **Virtual Portfolio:** A short, recorded video response (approxim